In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import datetime as dt
import os
import re


dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df_train = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
#open csv

df_test = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
weather_df = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))


In [2]:

df_train = df_train.drop(columns='Bump').drop(columns='Give_Way').drop(columns='No_Exit').drop(columns='Roundabout')
df_test = df_test.drop(columns='Bump').drop(columns='Give_Way').drop(columns='No_Exit').drop(columns='Roundabout')

#change true and false ->>> 0 and 1 
df_train["Crossing"] = df_train["Crossing"].astype(int)
df_train["Junction"] = df_train["Junction"].astype(int)
df_train["Railway"]  = df_train["Railway"].astype(int)
df_train["Stop"]     = df_train["Stop"].astype(int)
df_train["Amenity"]  = df_train["Amenity"].astype(int)

df_test["Crossing"] = df_test["Crossing"].astype(int)
df_test["Junction"] = df_test["Junction"].astype(int)
df_test["Railway"]  = df_test["Railway"].astype(int)
df_test["Stop"]     = df_test["Stop"].astype(int)
df_test["Amenity"]  = df_test["Amenity"].astype(int)

#change R and L ->>> 0 and 1
df_train["Side"].replace({"R": 1, "L": 0}, inplace=True)
df_test["Side"].replace({"R": 1, "L": 0}, inplace=True)

#read xml file and convert to list of Strings

tree = ET.parse((os.path.join(dataset_path, "holidays.xml") ))
root = tree.getroot()
holidays = []
for elem in root:
    holidays.append(elem[0].text)
#////////////////////////////////////////
df_holidays =pd.DataFrame(holidays)

#////////////////////////////////////////////////
df_train['timestamp'] =  pd.to_datetime(df_train['timestamp'], format='%Y-%m-%d %H:%M:%S')

# now splite th date into 2 columns but still strings 
df_train['Date'] = [d.date() for d in df_train['timestamp']]
df_train['time'] = [d.time() for d in df_train['timestamp']]
df_train['IsHoliday'] = np.where(df_train['Date'].apply(lambda x:True if str (x) in holidays else False ), 1, 0)
df_train['time']=df_train['time'].apply(lambda x: re.findall('\d\d', str(x))[0]).astype(int)
df_train=df_train.rename(columns={'time': 'Hour'})

# on test file also 
df_test['timestamp'] =  pd.to_datetime(df_test['timestamp'], format='%Y-%m-%d %H:%M:%S')

df_test['Date'] = [d.date() for d in df_test['timestamp']]
df_test['time'] = [d.time() for d in df_test['timestamp']]
df_test['IsHoliday'] = np.where(df_test['Date'].apply(lambda x:True if str (x) in holidays else False ), 1, 0)
df_test['time']=df_test['time'].apply(lambda x: re.findall('\d\d', str(x))[0]).astype(int)
df_test=df_test.rename(columns={'time': 'Hour'})

# drop the old column of timestamp 
df_train = df_train.drop(columns='timestamp')
df_test = df_test.drop(columns='timestamp')
#////////////////////////////////////////////////////////////////////////
# now change the value of hour to 3 category in train 
df_train.loc[ (df_train['Hour']>= 0) & (df_train['Hour']<=7) , 'Hour']   = 1
df_train.loc[ (df_train['Hour']>= 8) & (df_train['Hour']<=15) , 'Hour']  = 2
df_train.loc[ (df_train['Hour']>= 16) & (df_train['Hour']<=23) , 'Hour'] = 3
##############################################
# now change the value of hour to 3 category in test 
df_test.loc[ (df_test['Hour']>= 0) & (df_test['Hour']<=7) , 'Hour']   = 1
df_test.loc[ (df_test['Hour']>= 8) & (df_test['Hour']<=15) , 'Hour']  = 2
df_test.loc[ (df_test['Hour']>= 16) & (df_test['Hour']<=23) , 'Hour'] = 3


#////////////////////////////////////////////////////////////////////////
#change date to time stamp 
df_train["Date"] = pd.to_datetime(df_train["Date"])

df_test["Date"] = pd.to_datetime(df_test["Date"])

df_train = df_train.sort_values(by="Date")
df_test = df_test.sort_values(by="Date")

df_train.to_csv("weatrain.csv")
df_test.to_csv("weatest.csv")


In [3]:
df_test

,ID,Lat,Lng,Distance(mi),Crossing,Junction,Railway,Stop,Amenity,Side,Date,Hour,IsHoliday
1181,7588,37.769009,-122.405144,0.000,0,0,0,0,0,1,2016-03-23,3,0
251,6658,37.734650,-122.475113,0.010,1,0,0,0,0,0,2016-03-27,2,0
241,6648,37.775314,-122.406052,0.010,0,0,0,0,0,1,2016-03-28,1,0
1345,7752,37.773136,-122.471779,0.000,0,0,0,0,0,0,2016-03-29,1,0
427,6834,37.737186,-122.402534,0.000,0,0,0,0,0,1,2016-03-30,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,7494,37.751523,-122.402726,0.111,0,0,0,0,0,1,2020-12-30,3,0
821,7228,37.781821,-122.397061,0.325,1,0,0,0,0,1,2020-12-31,3,0
384,6791,37.766317,-122.405257,0.079,0,0,0,0,0,1,2020-12-31,1,0
742,7149,37.730059,-122.439598,0.028,0,0,0,0,0,1,2020-12-31,1,0


In [4]:
weather_df = weather_df.drop(columns=['Precipitation(in)', 'Selected'])

weather_df['Date'] = weather_df['Year'].astype(str) + '-' + weather_df['Month'].astype(str) + '-' + weather_df['Day'].astype(str)

weather_df["Date"] = pd.to_datetime(weather_df["Date"])

weather_df['Wind_Chill(F)'] = weather_df['Wind_Chill(F)'].fillna(weather_df['Wind_Chill(F)'].mean())

weather_df['Temperature(F)'] = weather_df['Temperature(F)'].fillna(weather_df['Temperature(F)'].mean())

weather_df['Humidity(%)'] = weather_df['Humidity(%)'].fillna(weather_df['Humidity(%)'].median())

weather_df['Wind_Speed(mph)'] = weather_df['Wind_Speed(mph)'].fillna(weather_df['Wind_Speed(mph)'].median())

weather_df['Visibility(mi)'] = weather_df['Visibility(mi)'].fillna(weather_df['Visibility(mi)'].mean())
weather_df = weather_df.sort_values(by="Date")
weather_df.to_csv("weefirst.csv")

# now change the value of hour to 3 category 
weather_df.loc[ (weather_df['Hour']>= 0) & (weather_df['Hour']<=7) , 'Hour']   = 1
weather_df.loc[ (weather_df['Hour']>= 8) & (weather_df['Hour']<=15) , 'Hour']  = 2
weather_df.loc[ (weather_df['Hour']>= 16) & (weather_df['Hour']<=23) , 'Hour'] = 3

#now work on Weather_Condition 


weight = 0.25
fair = ['Fair', 'Clear', 'Scattered Clouds']
med_fair = ['Partly Cloudy', 'Cloudy', 'Light Drizzle', 'Light Rain', 'Shallow Fog']
bad = ['Mostly Cloudy', 'Rain', 'Windy', 'Light Thunderstorms and Rain', 'Patches of Fog', 'Overcast']
worst = ['Haze', 'Heavy Rain', 'Fog', 'Smoke', 'Mist', 'Squalls']
complex_weather = {'Cloudy / Windy': 1.25,'Fair / Windy': 1,'Fog / Windy': 1.75,'Mostly Cloudy / Windy': 1.5,'Partly Cloudy / Windy': 1.25,'Rain / Windy':1.5, 'Light Rain / Windy': 1.25}

weather_df.loc[ weather_df['Weather_Condition'].isin (fair), 'Weather_Condition']   = 1 * weight
weather_df.loc[ weather_df['Weather_Condition'].isin (med_fair), 'Weather_Condition']   = 2 * weight
weather_df.loc[ weather_df['Weather_Condition'].isin (bad), 'Weather_Condition']   = 3 * weight
weather_df.loc[ weather_df['Weather_Condition'].isin (worst), 'Weather_Condition']   = 4 * weight
weather_df['Weather_Condition'].replace(complex_weather, inplace=True)
weather_df['Date']= weather_df['Date'].dt.strftime("%Y-%m-%d")

#//////////////////////////////////////////////////////////////////////////////////////////////
#now reduce rows of weather 

unique_Date = weather_df['Date'].unique()
Final_weather = pd.DataFrame(columns =weather_df.columns )
aggregations = {
    'Year':'first',
    'Day':'first',
    'Month':'first',
    'Weather_Condition':'mean',
    'Wind_Chill(F)':'mean',
    'Temperature(F)':'mean',
    'Humidity(%)':'mean',
    'Wind_Speed(mph)':'mean',
    'Humidity(%)':'mean',
    'Visibility(mi)':'mean',
    'Date':'first'
}

for day in unique_Date:
    df_single_day = weather_df[weather_df['Date']== day ]
    df_single_day=df_single_day.groupby(['Hour']).agg(aggregations).reset_index()
    Final_weather = Final_weather.append(df_single_day)

Final_weather = Final_weather.drop(columns='Year').drop(columns='Month').drop(columns='Day')

In [5]:
Final_weather['Hour']=Final_weather['Hour'].astype(int)
Final_weather["Date"] = pd.to_datetime(Final_weather["Date"])
Final_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3265 entries, 0 to 2
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Hour               3265 non-null   int64         
 1   Weather_Condition  3264 non-null   float64       
 2   Wind_Chill(F)      3265 non-null   float64       
 3   Temperature(F)     3265 non-null   float64       
 4   Humidity(%)        3265 non-null   float64       
 5   Wind_Speed(mph)    3265 non-null   float64       
 6   Visibility(mi)     3265 non-null   float64       
 7   Date               3265 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 229.6 KB


In [6]:
merg_test=pd.merge(df_test, Final_weather, how="left", on=["Date", "Hour"])
merg_train = pd.merge(df_train, Final_weather, how="left", on=["Date", "Hour"])
merg_train

,ID,Lat,Lng,Distance(mi),Crossing,Junction,Railway,Stop,Amenity,Side,Severity,Date,Hour,IsHoliday,Weather_Condition,Wind_Chill(F),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,3657,37.807910,-122.367400,0.000,0,1,0,0,0,1,2,2016-03-23,3,0,0.500000,59.762515,60.000000,64.500000,16.7,10.0
1,3724,37.779530,-122.400124,0.010,0,0,0,0,0,1,3,2016-03-23,2,0,0.750000,59.762515,55.450000,69.500000,10.0,10.0
2,6024,37.745080,-122.404930,0.150,0,1,0,0,0,1,2,2016-03-23,2,0,0.750000,59.762515,55.450000,69.500000,10.0,10.0
3,985,37.786060,-122.390900,0.039,0,1,0,0,0,1,2,2016-03-24,2,0,0.500000,59.762515,57.000000,75.333333,5.4,10.0
4,1960,37.785549,-122.391373,0.010,0,1,0,0,0,1,3,2016-03-24,2,0,0.500000,59.762515,57.000000,75.333333,5.4,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,401,37.814015,-122.477860,1.118,0,0,0,0,0,1,2,2020-12-31,1,0,0.250000,51.600000,51.600000,75.800000,10.8,9.0
6403,3832,37.766317,-122.405257,0.106,0,0,0,0,0,1,2,2020-12-31,1,0,0.250000,51.600000,51.600000,75.800000,10.8,9.0
6404,3152,37.765234,-122.404893,0.397,0,0,0,0,0,1,2,2020-12-31,1,0,0.250000,51.600000,51.600000,75.800000,10.8,9.0
6405,6375,37.737490,-122.402238,0.043,0,0,0,0,0,1,2,2020-12-31,2,0,0.458333,58.333333,58.333333,57.500000,11.0,10.0


In [7]:

print("The shape of the dataset is {}.\n\n".format(merg_train.shape))
print("The shape of the dataset is {}.\n\n".format(merg_test.shape))

merg_train['Date'] = merg_train['Date'].map(dt.datetime.toordinal)
merg_train.dropna(subset = ["Weather_Condition"], inplace=True)


merg_test['Date'] = merg_test['Date'].map(dt.datetime.toordinal)
merg_test.dropna(subset = ["Weather_Condition"], inplace=True)
merg_train

The shape of the dataset is (6407, 20).


The shape of the dataset is (1601, 19).




,ID,Lat,Lng,Distance(mi),Crossing,Junction,Railway,Stop,Amenity,Side,Severity,Date,Hour,IsHoliday,Weather_Condition,Wind_Chill(F),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,3657,37.807910,-122.367400,0.000,0,1,0,0,0,1,2,736046,3,0,0.500000,59.762515,60.000000,64.500000,16.7,10.0
1,3724,37.779530,-122.400124,0.010,0,0,0,0,0,1,3,736046,2,0,0.750000,59.762515,55.450000,69.500000,10.0,10.0
2,6024,37.745080,-122.404930,0.150,0,1,0,0,0,1,2,736046,2,0,0.750000,59.762515,55.450000,69.500000,10.0,10.0
3,985,37.786060,-122.390900,0.039,0,1,0,0,0,1,2,736047,2,0,0.500000,59.762515,57.000000,75.333333,5.4,10.0
4,1960,37.785549,-122.391373,0.010,0,1,0,0,0,1,3,736047,2,0,0.500000,59.762515,57.000000,75.333333,5.4,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,401,37.814015,-122.477860,1.118,0,0,0,0,0,1,2,737790,1,0,0.250000,51.600000,51.600000,75.800000,10.8,9.0
6403,3832,37.766317,-122.405257,0.106,0,0,0,0,0,1,2,737790,1,0,0.250000,51.600000,51.600000,75.800000,10.8,9.0
6404,3152,37.765234,-122.404893,0.397,0,0,0,0,0,1,2,737790,1,0,0.250000,51.600000,51.600000,75.800000,10.8,9.0
6405,6375,37.737490,-122.402238,0.043,0,0,0,0,0,1,2,737790,2,0,0.458333,58.333333,58.333333,57.500000,11.0,10.0


In [8]:
from sklearn.model_selection import train_test_split

df_train, val_df = train_test_split(merg_train, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = df_train.drop(columns=['ID', 'Severity'])
y_train = df_train['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']



In [9]:
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5124 entries, 748 to 860
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Lat                5124 non-null   float64
 1   Lng                5124 non-null   float64
 2   Distance(mi)       5124 non-null   float64
 3   Crossing           5124 non-null   int64  
 4   Junction           5124 non-null   int64  
 5   Railway            5124 non-null   int64  
 6   Stop               5124 non-null   int64  
 7   Amenity            5124 non-null   int64  
 8   Side               5124 non-null   int64  
 9   Date               5124 non-null   int64  
 10  Hour               5124 non-null   int64  
 11  IsHoliday          5124 non-null   int64  
 12  Weather_Condition  5124 non-null   float64
 13  Wind_Chill(F)      5124 non-null   float64
 14  Temperature(F)     5124 non-null   float64
 15  Humidity(%)        5124 non-null   float64
 16  Wind_Speed(mph)    5124

In [10]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train.to_frame())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


In [11]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.749609984399376


In [12]:
X_test = merg_test.drop(columns=['ID'])

y_test_predicted = classifier.predict(X_test)

merg_test['Severity'] = y_test_predicted

In [13]:
merg_test[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)